In [49]:
from transformers import pipeline
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct", device_map="cuda")

Device set to use cuda


In [50]:
from config import setting
from myprompt import zero_short_generation_prompt
setting.DEFULT_MODEL

'Qwen/Qwen2.5-0.5B-Instruct'

In [51]:
with open('data/amazone_banned_phrases.txt') as ifp:
    banned_phrases= [line.strip().lower() for line in ifp.readlines()]

banned_phrases[:5]


['% off', '100% natural', '100% quality guaranteed', '100% value', 'abuse']

In [52]:
with open('data/desired_phrases.txt') as ifp:
    desired_phrases= [line.strip().lower() for line in ifp.readlines()]

desired_phrases[:5]


['fairlife protein shake',
 'nutritional yeast',
 'premier protein shakes',
 'serious mass',
 'high calorie foods']

## make uniq

In [53]:
banned_phrases = set(banned_phrases)
desired_phrases= set(desired_phrases)
len(banned_phrases) , len(desired_phrases)

(411, 29)

### without custom logis , zero short generation

In [55]:
from config import setting
pipe= pipeline(
    task= 'text-generation',
    model= setting.DEFULT_MODEL,
    device_map="cuda",
    kwargs={
        'return_full_text':False ## true it return inpput + output , but we only want output so false
    },
    model_kwargs={}
    
)

Device set to use cuda


In [56]:
def generate_product_description(item:str) -> str:
    input_message= [
        {'role': 'system' , 'content' : zero_short_generation_prompt.system_prompt } , 
        {'role': 'user'  , 'content': zero_short_generation_prompt.user_prompt.format(item=item) }
    ]

    results= pipe(input_message, max_new_tokens= 512)
    return results


In [57]:
prod = generate_product_description("protein drink")
answer= prod[0]['generated_text'][-1]['content']

In [58]:
import textwrap ## for preety print this 
print(textwrap.fill(answer , width= 60))

Experience the ultimate boost in energy and muscle growth
with our innovative protein drink. Perfectly formulated to
provide the necessary amino acids for muscle repair and
growth, this drink is packed with essential vitamins and
minerals to support overall health and performance. Made
from premium ingredients, including whey protein powder and
other natural ingredients like ginger and turmeric, it
delivers a satisfying taste without any artificial flavors
or colors. Perfect for busy professionals looking for a
quick and effective way to fuel their workouts and maintain
their fitness goals.


### lets see how many banned words and prefered words does our llm had

In [59]:
def evaluate(descr: str,   positives, negatives):
    descr= descr.lower()
    num_positive  = num_negative= 0 
    for phrases in positives:
        if phrases in descr:
            num_positive +=1 
        
    for phrases in negatives:
        if phrases in descr:
                num_negative +=1 

    return num_positive  - num_negative

            


In [60]:
def evaluate_verboose(descr: str, positives, negatives)-> int: 
    descr= descr.lower()

    num_positive= num_negative = 0 
    for phrases in positives:
        if phrases in descr:
            num_positive +=1 
            print(f"Good: {phrases}")

    for phrases in negatives:
        if phrases in descr:
            num_negative += 1 
            print(f"Bad: {phrases}")
    print(f"Good: {num_positive} Bad {num_negative}")

    return num_positive - num_negative 



In [62]:
evaluate(descr= answer , 
positives = desired_phrases   , 
negatives= banned_phrases)

-7

In [63]:
evaluate_verboose(descr= answer , 
positives = desired_phrases   , 
negatives= banned_phrases)

Bad: natural
Bad: perfect for
Bad: perfect
Bad: als
Bad: ultimate
Bad: heal
Bad: growth
Good: 0 Bad 7


-7

## so prompts is not helping 

### now we do logits masking

# Step 1: Intercepting sampling

To intercept sampling and get access to the probability of each continuation
sequence, you need to create a LogitsProcessor subclass, which is
part of Transformers’ generation library.

In [64]:
import torch
import numpy as np
from transformers.generation.logits_process import (
    LogitsProcessor,
    LOGITS_PROCESSOR_INPUTS_DOCSTRING,
)
from transformers.utils import add_start_docstrings

In [102]:
class BrandLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer ,  positives,  negatives):
        self.tokenizer= tokenizer 
        self.positives= positives 
        self.negatives= negatives

    def __call__(self, inputs_ids: torch.LongTensor, 
                        input_logits: torch.FloatTensor): ### these are model internal tokens (i.e everything model thinks about) and logits 

        output_logits = input_logits.clone()
        num_matches= [0] * len(inputs_ids)

        for idx, seq in enumerate(inputs_ids):
            ## decode the sequence 
            decoded= self.tokenizer.decode(seq) ## truns numer into text 
            num_matches[idx] = evaluate(decoded , self.positives, self.negatives)

        max_matches= np.max(num_matches)

        for idx in range(len(inputs_ids)):
            if num_matches[idx] != max_matches:
                output_logits[idx] = -1000 

        return output_logits
            


In [103]:
inpu

NameError: name 'inpu' is not defined

In [113]:
def generate_product_description_v2(item:str) -> str:
    input_message =  [
        {'role': 'system', 'content': zero_short_generation_prompt.system_prompt},
        {'role':'user', 'content': zero_short_generation_prompt.user_prompt.format(item= item)}
    ]
    pipe.model = pipe.model.to("cpu") ## no gpu sad
    pipe.device = torch.device("cpu") # make pipeline aware it's on CPU    brand_processor=  BrandLogitsProcessor(pipe.tokenizer , desired_phrases,  banned_phrases)
    brand_processor= BrandLogitsProcessor(pipe.tokenizer, desired_phrases, banned_phrases)
    results=  pipe(input_message,
                        max_new_tokens=300,
                        temperature=.8,
                        num_beams=10,
                        logits_processor= [brand_processor])

    return results[0]['generated_text'][-1]['content'].strip()


prod = generate_product_description_v2("protein drink")
print(prod)



Elevate your energy levels with our high-protein protein drink. Rich in essential amino acids, this drink provides sustained energy throughout the day. Enjoy smoothies, workouts, or any other time when you need a boost of protein.


In [ ]:
prod = generate_product_description_v2("protein drink")


In [114]:
textwrap.wrap(prod)

['Elevate your energy levels with our high-protein protein drink. Rich',
 'in essential amino acids, this drink provides sustained energy',
 'throughout the day. Enjoy smoothies, workouts, or any other time when',
 'you need a boost of protein.']

In [115]:
evaluate_verboose(prod, desired_phrases , banned_phrases)

Good: 0 Bad 0


0

In [110]:
textwrap.wrap(answer)

['Experience the ultimate boost in energy and muscle growth with our',
 'innovative protein drink. Perfectly formulated to provide the',
 'necessary amino acids for muscle repair and growth, this drink is',
 'packed with essential vitamins and minerals to support overall health',
 'and performance. Made from premium ingredients, including whey protein',
 'powder and other natural ingredients like ginger and turmeric, it',
 'delivers a satisfying taste without any artificial flavors or colors.',
 'Perfect for busy professionals looking for a quick and effective way',
 'to fuel their workouts and maintain their fitness goals.']